In [ ]:
path = "./2.jpg"
threshold = 1 #1
threshold2 = 9#9
K=15 #15

In [ ]:
from PIL import Image
import cv2 as cv
import numpy as np
import operator
from collections import deque
from io import StringIO

In [ ]:
def add_tuple(a, b):
    return tuple(map(operator.add, a, b))

def sub_tuple(a, b):
    return tuple(map(operator.sub, a, b))

# def neg_tuple(a):
#     return tuple(map(operator.neg, a))

def direction(edge):
    return sub_tuple(edge[1], edge[0])

def magnitude(a):
    return int(pow(pow(a[0], 2) + pow(a[1], 2), .5))

def normalize(a):
    mag = magnitude(a)
    assert mag > 0, "Cannot normalize a zero-length vector"
    return tuple(map(operator.truediv, a, [mag]*len(a)))

def svg_header(width, height):
    return """<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg width="%d" height="%d"
     xmlns="http://www.w3.org/2000/svg" version="1.1">
""" % (width, height)

def joined_edges(assorted_edges, keep_every_point=False):
    pieces = []
    piece = []
    directions = deque([
        (0, 1),
        (1, 0),
        (0, -1),
        (-1, 0),
        ])
    while assorted_edges:
        if not piece:
            piece.append(assorted_edges.pop())
        current_direction = normalize(direction(piece[-1]))
        while current_direction != directions[2]:
            directions.rotate()
        for i in range(1, 4):
            next_end = add_tuple(piece[-1][1], directions[i])
            next_edge = (piece[-1][1], next_end)
            if next_edge in assorted_edges:
                assorted_edges.remove(next_edge)
                if i == 2 and not keep_every_point:
                    # same direction
                    piece[-1] = (piece[-1][0], next_edge[1])
                else:
                    piece.append(next_edge)
                if piece[0][0] == piece[-1][1]:
                    if not keep_every_point and normalize(direction(piece[0])) == normalize(direction(piece[-1])):
                        piece[-1] = (piece[-1][0], piece.pop(0)[1])
                        # same direction
                    pieces.append(piece)
                    piece = []
                break
        else:
            raise Exception ("Failed to find connecting edge")
    return pieces

In [ ]:
image = cv.imread(path)

#blur image
blur = cv.fastNlMeansDenoisingColored(image,None,10,10,7,21)


#convert to LAB
lab = cv.cvtColor(blur, cv.COLOR_BGR2Lab)

#do k means for color clustering
Z = lab.reshape((-1,3))
Z = np.float32(Z)
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center=cv.kmeans(Z,K,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((lab.shape))

#convert back to RGB
image= cv.cvtColor(res2, cv.COLOR_Lab2BGR)

#saving the image as refined image in same folder
cv.imwrite("./refined.png", image)

True

In [ ]:
main_cluster_list = []
small_cluster = []


# image = cv.imread(path)
# im = np.array(image,dtype = 'int64')
image = Image.open("./refined.png").convert('RGBA')
im = np.array(image,dtype='int64')
width = im.shape[0]
height = im.shape[1]

adjacent = ((1, 0), (0, 1), (-1, 0), (0, -1))#,(-1,-1),(1,1),(1,-1),(-1,1))
visited = np.zeros((width,height),dtype = int)
for x in range(width):
    for y in range(height):
        here = (x,y)
        if visited[here]:
            continue
        rgb = im[x, y]
        piece = []
        queue = [here]
        visited[here] = 1
        while queue:
            here = queue.pop()
            rgb = im[here]
            for offset in adjacent:
                neighbour = (here[0]+offset[0],here[1]+offset[1])
                if not (0 <= neighbour[0] < width) or not (0 <= neighbour[1] < height):
                    continue
                neighbour_rgba = im[neighbour]
                t = (neighbour_rgba[0]-rgb[0])/4+(neighbour_rgba[1]-rgb[1])/4+(neighbour_rgba[2]-rgb[2])/4+(neighbour_rgba[3]-rgb[3])/4
                if t>=threshold or -t>=threshold:
                    continue
                im[neighbour]=rgb
                if visited[neighbour]:
                    continue
                queue.append(neighbour)
                visited[neighbour] = 1
            piece.append((here[1],here[0]))
        if(len(piece)>threshold2):
            main_cluster_list.append(piece)
        else:
            small_cluster.append(piece)

#declaring array to number the pixels in the cluster and using 4th index as id cluter_id intializing all ids to zeros

arr = np.zeros((width,height,5),dtype=int)

for x in range(width):
    for y in range(height):
        arr[x,y,0] = im[x,y,0]
        arr[x,y,1] = im[x,y,1]
        arr[x,y,2] = im[x,y,2]
        arr[x,y,3] = im[x,y,3]


#giving cluster_id to pixels in main cluster and saving colors for those cluster id's
cluster_index_list = {}
color_index_list = {}
i=1
for piece in main_cluster_list:
    cluster_index_list[i] = piece
    color = [0,0,0,0]
    for y,x  in piece:
        color = im[x,y]
        arr[x,y,4] = i
    color_index_list[i] = (color[0],color[1],color[2],color[3])
    i = i+1

visited = np.zeros((width,height),dtype = int)

for piece in small_cluster:
    #find nearest clusters and nearest color in that cluster by bfs
    val = [260,260,260,260]
    dist = 600
    id=0
    for cord in piece:
        rgb = im[cord[1],cord[0]]
        for offset in adjacent:
            neighbour = (cord[1]+offset[0], cord[0]+offset[1])
            if not (0 <= neighbour[0] < width) or not (0 <= neighbour[1] < height):
                continue
            if (arr[neighbour[0],neighbour[1],4]==0):
                continue
            neighbour_rgba = im[neighbour]
            dist1 = pow((pow(neighbour_rgba[0]-rgb[0],2)+pow(neighbour_rgba[1]-rgb[1],2)+pow(neighbour_rgba[2]-rgb[2],2)+pow(neighbour_rgba[3]-rgb[3],2)),0.5)
            if dist1<dist:
                dist = dist1
                val[0]= rgb[0]
                val[1]=rgb[1]
                val[2]=rgb[2]
                val[3]=rgb[3]
                id = arr[neighbour[0],neighbour[1],4]

    #add these pixels to that clusters id
    if id!=0:
        for cord in piece:
            cluster_index_list[id].append(cord)
            arr[cord[1],cord[0],4] = id

m=0
for x in range(arr.shape[0]):
    for y in range(arr.shape[1]):
        if arr[x,y,3]==0:
            m=m+1

print(m,"\n")

# print(len(main_cluster_list))

0 

165


In [ ]:
color_pixel_lists = {}
for id in cluster_index_list:
    rgb = tuple(color_index_list[id])
    if not rgb in color_pixel_lists:
        color_pixel_lists[rgb] = []
    color_pixel_lists[rgb].append(cluster_index_list[id])

# print(color_pixel_lists)

In [ ]:
edges = {
        (-1, 0):((0, 0), (0, 1)),
        (0, 1):((0, 1), (1, 1)),
        (1, 0):((1, 1), (1, 0)),
        (0, -1):((1, 0), (0, 0)),
        }

color_edge_lists = {}

for rgba, pieces in color_pixel_lists.items():
    for piece_pixel_list in pieces:
        edge_set = set([])
        for coord in piece_pixel_list:
            for offset, (start_offset, end_offset) in edges.items():
                neighbour = add_tuple(coord, offset)
                start = add_tuple(coord, start_offset)
                end = add_tuple(coord, end_offset)
                edge = (start, end)
                if neighbour in piece_pixel_list:
                    continue
                edge_set.add(edge)
        if not rgba in color_edge_lists:
            color_edge_lists[rgba] = []
        color_edge_lists[rgba].append(edge_set)

print("edges extraction from points done \n")
print(color_edge_lists)

color_joined_pieces = {}

for color, pieces in color_edge_lists.items():
    color_joined_pieces[color] = []
    for assorted_edges in pieces:
        print(assorted_edges,"\n")
        color_joined_pieces[color].append(joined_edges(assorted_edges, False))

print("joining edges done \n")

s = StringIO()
s.write(svg_header(*image.size))



for color, shapes in color_joined_pieces.items():
    for shape in shapes:
        s.write(""" <path d=" """)
        for sub_shape in shape:
            here = sub_shape.pop(0)[0]
            start = [here[0],here[1]]
            s.write(""" M %d,%d """ % here)
            prepre=[here[0],here[1]]
            pre=[0,0]
            m=0
            for edge in sub_shape:
                # here = edge[0]
                # s.write(""" L %d,%d """ % here)
                here = edge[0]
                if m==0:
                    m=1
                else:
                    s.write(""" L %d,%d L %d,%d L %d,%d L %d,%d""" % (here[0],here[1],prepre[0],prepre[1],pre[0],pre[1],here[0],here[1]))
                    prepre[0]=pre[0]
                    prepre[1]=pre[1]
                pre[0]=here[0]
                pre[1]=here[1]
            s.write(""" L %d,%d L %d,%d L %d,%d L %d,%d""" % (start[0],start[1],prepre[0],prepre[1],pre[0],pre[1],start[0],start[1]))
            s.write(""" Z """)
        s.write(""" " style="fill:rgb%s; fill-opacity:%.3f; stroke:none;" />\n""" % (color[0:3], float(color[3]) / 255))
print("creating paths done \n")
s.write("""</svg>\n""")
with open("output_man.svg", "w") as text_file:
    text_file.write(s.getvalue())